Initial data checks

Changing data type

Check for mixed data

Look for missing observations

Create a new dataframe with null values removed

Find duplicate rows



In [ ]:
## -- Loading Libraries --
import pandas as pd
import numpy as np
import os

In [ ]:
#Importing the wrangled orders df
df_ords = pd.read_csv(r'/content/orders_wrangled.csv', index_col = False)

**Initial data checks**

In [ ]:
#Checking orders df look as expected

df_ords.head()

,Unnamed: 0,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
0,0,2539329,1,prior,1,2,8,NaN
1,1,2398795,1,prior,2,3,7,15.0
2,2,473747,1,prior,3,3,12,21.0
3,3,2254736,1,prior,4,4,7,29.0
4,4,431534,1,prior,5,4,15,28.0


In [ ]:
#Checking data within in df_ords

df_ords.describe()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,138594.000000,1.385940e+05,138594.000000,138594.000000,138594.000000,138594.000000,130256.000000
mean,69296.500000,1.710601e+06,4221.438093,17.280221,2.795871,13.451138,11.039883
std,40008.785941,9.889246e+05,2401.868257,17.934762,2.045520,4.206474,9.178217
min,0.000000,8.000000e+00,1.000000,1.000000,0.000000,0.000000,0.000000
25%,34648.250000,8.551818e+05,2157.000000,5.000000,1.000000,10.000000,4.000000
50%,69296.500000,1.710074e+06,4286.500000,11.000000,3.000000,13.000000,7.000000
75%,103944.750000,2.566776e+06,6292.000000,24.000000,5.000000,16.000000,15.000000
max,138593.000000,3.421068e+06,8338.000000,100.000000,6.000000,23.000000,30.000000


**Changing data type**

In [ ]:
#Setting the order_id and user_id as string type - beacuse their not numeric values, just descriptions

df_ords['order_id'] = df_ords['order_id'].astype('str')
df_ords['user_id'] = df_ords['user_id'].astype('str')

In [ ]:
#Checking data within in df_ords

df_ords.describe()

,Unnamed: 0,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,138594.000000,138594.000000,138594.000000,138594.000000,130256.000000
mean,69296.500000,17.280221,2.795871,13.451138,11.039883
std,40008.785941,17.934762,2.045520,4.206474,9.178217
min,0.000000,1.000000,0.000000,0.000000,0.000000
25%,34648.250000,5.000000,1.000000,10.000000,4.000000
50%,69296.500000,11.000000,3.000000,13.000000,7.000000
75%,103944.750000,24.000000,5.000000,16.000000,15.000000
max,138593.000000,100.000000,6.000000,23.000000,30.000000


Days since prior order only goes up to 30, implying data may only be 30 days old. OR it's maybe only looking at days in the month since last order, not looking at the months as well

Would need to look at how this data is being collected, and ask more questions

**Check for mixed data**

In [ ]:
#CHecking for mixed data types

for col in df_ords.columns:
    col_series = df_ords[col]
    col_type = type(col_series.iloc[0])
    weird = col_series.map(type) != col_type
    if weird.any():
        print(col)

Unnamed: 0
order_number
orders_day_of_week
order_hour_of_day
days_since_prior_order


This result shows all columns have a mixed data type, which I was not expecting. Going to revert to using the given code, even though this comes with a warning

In [ ]:
#Checking for mixed data types

for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)

/tmp/ipython-input-32-932771391.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
/tmp/ipython-input-32-932771391.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
/tmp/ipython-input-32-932771391.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
/tmp/ipython-input-32-932771391.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
/tmp/ipython-input-32-932771391.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_ords[[col]

But that result does not help!

So, i'm going to assume it's correct that they all return mixed types and correct as I see fit

This obviously poroduces an error, and the explanation is:
"The error IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer occurs because you are trying to convert the days_since_prior_order column, which contains missing values (NaN), to an integer type (int64). Integer types cannot hold missing values like NaN.

To fix this, you can replace the missing values in the days_since_prior_order column with a suitable value, such as 0, before converting the column to an integer type."

So, I need to look again at the days_since_prior_order column first

**Look for missing observations**

In [ ]:
#Checking how many missing observations (cells/datapoints) there are in df_ords

df_ords.isnull().sum()

,0
Unnamed: 0,0
order_id,0
user_id,0
eval_set,0
order_number,0
orders_day_of_week,0
order_hour_of_day,0
days_since_prior_order,8338


This is useful - it showing again that the days_since_prior_order column is where the issue is

So perhaps we've lost data of over 30 days old - which would be consistent with their being a missing "months" data column. Perhaps it wasn't set up to record anything over 30 days.

In [ ]:
# Investigating values in days_since_prior_order

df_ords['days_since_prior_order'].value_counts(dropna = False)

,count
days_since_prior_order,
30.0,14712
7.0,12996
6.0,9593
3.0,8985
4.0,8964
5.0,8616
NaN,8338
2.0,8202
8.0,7250


In [ ]:
## Looking at total number of rows

df_ords.shape

(138594, 8)

So, we're looking at c8,000 rows out of c138,000, so we can just create a subset without those

(An alternative would be to add another column of "Known result and "Not Known Result" and then create a subset that doesn't include the "Not Known Result" but I don't yet know how to do that!)

**Create a new dataframe with null values removed**

In [ ]:
#Creates a new dataframe with all of the null values removed - i.e. removing the whole row

df_ords_clean = df_ords[df_ords['days_since_prior_order'].isnull() == False]

In [ ]:
## Looking at total number of rows

df_ords_clean.shape

(130256, 8)

We can see we've removed about 8,000 - so all is good!

**Find duplicate rows**

In [ ]:
# Finding duplicates

## looking for rows that have identical data

df_ords_dups = df_ords_clean[df_ords_clean.duplicated()]

In [ ]:
#seeing what we have

df_ords_dups

,Unnamed: 0,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order


Don't have any duplicate values, which I perhaps didn't expect, but perhaps the cleaning we've already done has taken care of that. I can check that by looking at the original df

In [ ]:
# Finding duplicates

## looking for rows that have identical data

df_ords_dups_unclean = df_ords[df_ords.duplicated()]

In [ ]:
#seeing what we have

df_ords_dups_unclean

,Unnamed: 0,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order


Still no duplicates, so was clean of duplicates all along

In [ ]:
#Export our clean data

df_ords_clean.to_csv(os.path.join(r'/content/', 'orders_checked.csv'))